In [5]:
from pyomo.environ import *
import pandas as pd

In [6]:
# import data

gen = pd.read_excel('3bus_data_lecture.xlsx',sheet_name='gen',index_col='GeneratorName')
line = pd.read_excel('3bus_data_lecture.xlsx','line',index_col='lineID')
demand = pd.read_excel('3bus_data_lecture.xlsx','demand',index_col='DemandID')


# gen = pd.read_excel('5bus_data.xlsx',sheet_name='gen',index_col='GeneratorName')
# line = pd.read_excel('5bus_data.xlsx','line',index_col='lineID')
# demand = pd.read_excel('5bus_data.xlsx','demand',index_col='DemandID')

In [7]:
gen

,bus,Capacity,Cost
GeneratorName,,,
A,1,140,7.5
B,1,285,6.0
C,2,90,14.0
D,3,85,10.0


In [8]:
# create sets, iterable objects
gens = gen.index #.tolist()
lines = line.index #.tolist()
demands = demand.index #.tolist()
buses = gen['bus'].unique()#.tolist()
# buses= ['A','B','C','D','E']

In [9]:
# Create Model
m = ConcreteModel()

m.dual = Suffix(direction=Suffix.IMPORT)

In [10]:
# Create Variables
m.power = Var(gens, domain=NonNegativeReals) # did not find a good syntax for bounds, have to use constraints. 
m.flow = Var(lines)
m.theta = Var(buses)


In [11]:
# Create Objective Function
exp_obj = sum([m.power[g]*gen.loc[g,'Cost'] for g in gens])

In [12]:
print(exp_obj)

7.5*power[A] + 6.0*power[B] + 14.0*power[C] + 10.0*power[D]


In [13]:
m.cost = Objective(expr=exp_obj, sense=minimize)

In [14]:
# Create Bus Balance Constraints
m.bus_bal = ConstraintList()
for i in buses:
    expr = (sum([m.power[g] for g in gen[gen['bus']==i].index])
             -sum([demand['MW'][d] for d in demand[demand['bus']==i].index])
             -sum([m.flow[k] for k in line[line['from']==i].index])
             +sum([m.flow[k] for k in line[line['to']==i].index]))
    print(expr)
    m.bus_bal.add(expr==0)
    


power[A] + power[B] - 50 - (flow[L12] + flow[L13])
power[C] - 60 - flow[L23] + flow[L12]
power[D] - 300 + flow[L13] + flow[L23]


In [15]:
# Create Flow equation constraint
m.flow_eq = ConstraintList()
for k in lines:
    flow_eq = (m.flow[k] == (round(1/line['reactance'][k],2)*(m.theta[line['from'][k]]-m.theta[line['to'][k]])))
    print(flow_eq)
    m.flow_eq.add(flow_eq)

# m.flow_eq = Constraint(Any)
# for k in lines:
#     m.flow_eq[k] = (m.flow[k] == (round(1/line['reactance'][k],2)*(m.theta[line['from'][k]]-m.theta[line['to'][k]])))
#     print(m.flow_eq[k])


flow[L12]  ==  5.0*(theta[1] - theta[2])
flow[L13]  ==  5.0*(theta[1] - theta[3])
flow[L23]  ==  10.0*(theta[2] - theta[3])


In [16]:
# Create Flow limit constraint
m.flow_limit = ConstraintList()
for k in lines:
    m.flow_limit.add(inequality(-line['capacity'][k], m.flow[k], line['capacity'][k]))
    #m.flow_limit.add(-line['capacity'][k]<=m.flow[k]<=line['capacity'][k])

In [17]:
# Designate a Reference bus
m.refBus = ConstraintList()
expr = (m.theta[1]==0)  
#expr = (m.theta['A']==0)
m.refBus.add(expr)


In [18]:
# Create Generator Capacity Limit Constraint

In [19]:
m.gen_cap = ConstraintList()
for g in gens:
    expr = m.power[g]<=gen['Capacity'][g]
    print(expr)
    m.gen_cap.add(expr)

power[A]  <=  140.0
power[B]  <=  285.0
power[C]  <=  90.0
power[D]  <=  85.0


In [20]:
SolverFactory('glpk').solve(m)

{'Problem': [{'Name': 'unknown', 'Lower bound': 2835.0, 'Upper bound': 2835.0, 'Number of objectives': 1, 'Number of constraints': 18, 'Number of variables': 11, 'Number of nonzeros': 31, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.019411802291870117}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [21]:
m.display()

Model unknown

  Variables:
    power : Size=4, Index=power_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :  50.0 :  None : False : False : NonNegativeReals
          B :     0 : 285.0 :  None : False : False : NonNegativeReals
          C :     0 :   0.0 :  None : False : False : NonNegativeReals
          D :     0 :  75.0 :  None : False : False : NonNegativeReals
    flow : Size=3, Index=flow_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
        L12 :  None : 126.0 :  None : False : False :  Reals
        L13 :  None : 159.0 :  None : False : False :  Reals
        L23 :  None :  66.0 :  None : False : False :  Reals
    theta : Size=3, Index=theta_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  None :   0.0 :  None : False : False :  Reals
          2 :  None : -25.2 :  None : False : False :  Reals
          3 :  None : -31.8 :  None : False : False :  Reals

  Objectives:
    cost : S

In [22]:
m.dual[m.bus_bal[2]]

11.25